

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_UR_NEWS.ipynb)






## **News Classifier for Urdu texts**


## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.1

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
!pip install --ignore-installed spark-nlp-display

## 2. Start Spark Session

In [ ]:
spark = sparknlp.start()

## 3. Sample Examples for Urdu texts.

In [ ]:
text_list=["""اقوام متحدہ کی سلامتی کونسل نے پیر کو آئیوری کوسٹ پر فوری طور پر ہتھیاروں کی پابندی عائد کر دی اور اہم حکومت اور باغی رہنماؤں کو اگلے ماہ اضافی پابندیوں کے ساتھ سزا دینے کے لیے ووٹ دیا۔""",
           """ اداکارہ نے سوچا تھا کہ برازیل کے کامیاب فٹ بالر نیمار بھی ان کی فلم کا حصہ بن جائیں گےفٹ بال کی دنیا میں بہترین کھلاڑی سمجھے جانے والے نیمار بھی دپیکا کے ساتھ اس ہولی وڈ فلم میں ڈیبیو کرنے جارہے ہیں اور انہوں نے""",
           """بیرون ملک تعلیم حاصل کرنا چاہتے ہیں مگر اتنے اخراجات کے لیے پیسے نہیںاگر ہاں تو اچھی بات یہ ہے کہ گھر بیٹھے بھی دنیا کی مشہور یونیورسٹیوں کے مختلف کورسز مفت پڑھ سکتے ہیں بس انٹرنیٹ کی ضرورت ہوگیطبی """,
           """کیا ایسے سپرسونک طیارے میں سفر کرنا پسند کریں گے جو کہ کسی ہیلی کاپٹر کی طرح کسی بھی جگہ لینڈنگ یا ٹیک کی صلاحیت رکھتا ہو جبکہ اسے چلانے کے لیے روایتی ایندھن کی بجائے بجلی کی ضرورت ہواسپیس ایکس اور""",
           """بوسٹن 18 مئی 2020کیا کوئی ماسک ازخود کورونا وائرس کی نشاندہی کرسکتا ہے اس دلچسپ سوال کا جواب اثبات میں ہے اور اس ایجاد کا سہرا میساچیوسیٹس انسٹی ٹیوٹ اف ٹیکنالوجی ایم ائی ٹی اور ہارورڈ یونیورسٹی کے سر""",
           """ ہوگئی ہے سات مہینوں میں ٹیکس مدنی 1ہزار 16 ارب روپے رہی جبکہ ٹیکس کے علاوہ دیگر مدنی 520 ارب روپے رہی ہے اعداد شمار کے مطابق ٹیکس کے علاوہ مدنی پچھلے سال سے 93 فیصد زیادہ ہے حکومت نے ان سات مہینوں میں 689 ارب روپے کی لاگت سے قرضوں پر سود ادا کیا ہے جبکہ پچھلے سال اس مد میں 576 ارب روپے خرچ ہوئے تھے جولائی سے جنوری کے دوران صوبوں نے اپنی مدنی سے 82 ارب روپے کم اخراجات کیے""",
           """شارجہپاکستان سپر لیگ کے گیارہویں میچ میں کوئٹہ گلیڈی ایٹر نے اسلام اباد یونائیٹڈ کے خلاف ٹاس جیت کر فیلنڈگ کرنے کا فیصلہ کیااسلام اباد یونائیٹڈ کی ٹیم میں کپتان شین واٹسن کے علاوہ شرجیل خان بریڈ ہیڈن خالد""",
           """ڈاکٹرمنصور نے یونیورسٹی آف میری لینڈ میڈیکل سینٹرمیں ڈاکٹرز کی ٹیم کے ساتھ مل کرایک مریض میں جینیاتی طورپرتبدیل شدہ خنزیز کا دل لگا دیا۔"""]

## 4. Define Spark NLP pipeline


In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document")

embeddings = BertSentenceEmbeddings.pretrained("labse", "xx") \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLModel.pretrained("classifierdl_bert_news", "ur") \
      .setInputCols(["document", "sentence_embeddings"]) \
      .setOutputCol("class_")

urdu_news_pipeline = Pipeline(
    stages=[
        document_assembler, 
        embeddings, 
        classifierdl])


labse download started this may take some time.
Approximate size to download 1.7 GB
[OK!]
classifierdl_bert_news download started this may take some time.
Approximate size to download 22.5 MB
[OK!]


## 5. Run the pipeline

In [ ]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = urdu_news_pipeline.fit(df).transform(df)

## 6. Visualize results


In [ ]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|document                                                                                                                                                                                                                                                                                                                                                                              |class        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------